# REQUISIÇÃO NA TWITER API


Nesta aba faremos uma requisição na API do twitter para obter os dados que serão usados neste trabalho.
Para isso usaremos a biblioteca *tweepy* para estabelecer comunicação com a API, e a biblioteca *pandas* para estruturar os dados recebidos.
Os dados extraidos são referentes aos tweets que citem a empresa *iFood* e aos ususarios que fizeram tais citações.
Os detalhes estarão inclusos no código como comentarios(tudo que preceder o simbolo *#*)

In [ ]:
# Importando as bibliotecas 
import pandas as pd
import tweepy

# Abrindo o arquivo com as credenciais de acesso a API do Twitter
credentials = open(r'C:\Users\gabri\repos\requisição API twitter\credentials.txt')
credentials = credentials.read().split()

consumer_key = credentials[0]
consumer_secret = credentials[1]
access_token = credentials[2]
access_token_secret = credentials[3]
bearer_token = credentials[4]

# Instanciando o objeto Client da biblioteca tweepy e passando as credenciais para acesso
client = tweepy.Client(consumer_key = consumer_key,
            consumer_secret = consumer_secret,
            access_token = access_token,
            access_token_secret = access_token_secret,
            bearer_token = bearer_token,
            return_type = dict(), # Solicitando o retorno em formato dicionario python
            wait_on_rate_limit = True) # Evitando a limitação da API

# Requisitando os dados para tweets
try:
    
    tweets_data = []
    
    # Procurando por tweets que contenham a palavra iFood e estejam em português
    query = '(iFood OR IFOOD OR IFood OR Ifood OR ifood) lang:pt'
    
    # Iterando sobre o metodo Paginator para obter os dados
    for tweet in tweepy.Paginator(client.search_recent_tweets, # Metodo que acessa dados dos ultimos 7 dias
                                  query = query,
                                  max_results = 100, # Maximo de resultados por requisição
                                  tweet_fields = ['created_at','author_id','public_metrics'],
                                  expansions = ['author_id','geo.place_id'],
                                 user_auth = True).flatten(limit = 10e4): # Maximo de tweets
        
        # Para cada objeto tweet retornado acessamos o dicionario com os dados
        temp = tweet.data
        
        # Separando apenas os dados de interesse
        twt_date = temp['created_at']
        retweet_count = temp['public_metrics']['retweet_count']
        twt = temp['text']
        like_count = temp['public_metrics']['like_count']
        user = temp['author_id']
        
        # Salvando os dados em um novo dict
        temp_dict = { "twt_date" : twt_date,
                      "retweet_count" : retweet_count,
                      "like_count" : like_count,
                      "tweet" : twt,
                      "user_id" : user
                    }
        # Metodo .append para unir os dados em cada iteração a uma mesma lista
        tweets_data.append(temp_dict)

# Mostrando erros caso ocorram
except tweepy.errors.TweepyException as e:
        print(e)

# Armazenando os dados obtidos em um pandas.DataFrame
tweets = pd.DataFrame(tweets_data)
        
# Requisição para os dados dos usuarios

# Separando os ids de cada ussuario para extração dos dados e removendo duplicatas
users_id = [int(user) for user in list(set(tweets['user_id']))]
# Para burlar uma limitação da biblioteca tweepy separamos nossa lista inicial em uma lista de listas
n = 171
users_id_splited = [users_id[i::n] for i in range(n)]

try:

    users_data = []

    # Iterando sobre cada lista com ids de usuarios
    for user_100 in users_id_splited:
    
        temp = client.get_users(ids = user_100,
                                user_fields = ['username','id','verified','public_metrics'])
        # Iterando sobre cada elemento da sublista
        for idx, name in enumerate(temp.data):
            
            # Separando os dados de interesse
            user_id = temp.data[idx]['id']
            screen_name = temp.data[idx]['username']
            verified = temp.data[idx]['verified']
            followers_count = temp.data[idx]['public_metrics']['followers_count']

            user_data = {'user_id' : user_id,
                         'screen_name' : screen_name,
                         'verified' : verified,
                         'followers_count' : followers_count
                        }
            users_data.append(user_data)

# Mostrando erros caso ocorram
except tweepy.errors.TweepyException as e:
    print(e)
    
users_data = pd.DataFrame(users_data)

# Exportando os dados para arquivos .csv
tweets.to_csv('tweets.csv')
users_data.to_csv('users_data.csv')